In [1]:
import tensorflow as tf
import numpy as np

# U-Net Modeling

In [2]:
def cache_for_conv_image_on_encoding():
    return ret


def conv_step(input_img, kernel, out_dim, strides=[1,1,1,1]):
    conv_filter1 = tf.Variable(tf.truncated_normal([kernel, kernel, int(input_img.get_shape()[-1]), out_dim], stddev=0.5, seed=1)) #weight dimensions.
    conv1 = tf.nn.conv2d(input=input_img, filter=conv_filter1, strides=strides, padding="VALID")
    
    conv_filter2 = tf.Variable(tf.truncated_normal([kernel, kernel, int(conv1.get_shape()[-1]), out_dim], stddev=0.5))
    conv2 = tf.nn.conv2d(input=conv1, filter=conv_filter2, strides=strides, padding="VALID")
    
    print("conv1 : ", conv1)
    print("conv2 : ", conv2)
    
    conv2 = tf.nn.relu(conv2)
    return conv2

def segmentation(input, kernel, out_dim, strides=[1,1,1,1]):
    conv_filter1 = tf.Variable(tf.truncated_normal([kernel, kernel, int(input_img.get_shape()[-1]), out_dim], stddev=0.5, seed=1)) #weight dimensions.
    conv1 = tf.nn.conv2d(input=input_img, filter=conv_filter1, strides=strides, padding="VALID")
    
    print("conv1 : ", conv1)
    conv2 = tf.nn.relu(conv2)
    return conv2
'''
def up_conv(input_img, kernel, out_dim, strides=[1,1,1,1]):
    up_convolution_filter = tf.Variable(tf.truncated_normal([kernel, kernel, int(input_img.get_shape()[-1]), out_dim], stddev=0.5))
    output_shape = [tf.shape(input_img)[0], int(input_img.get_shape()[1])*2, int(input_img.get_shape()[2]*2),out_dim]
    upsample = tf.nn.conv2d_transpose(value=input_img, filter=up_convolution_filter, output_shape=tf.stack(output_shape),strides=strides, padding="SAME")
    
    print("up-conv : ", upsample)
    return upsample
'''
def up_conv(input_img, kernel, out_dim, strides=[2,2]):
    up_convolution_filter = tf.Variable(tf.truncated_normal([kernel, kernel, int(input_img.get_shape()[-1]), out_dim], stddev=0.5))
    upsample = tf.layers.conv2d_transpose(input_img, filters=out_dim, kernel_size=(2,2), strides=strides, padding="SAME")
    
    print("up-conv : ", upsample)
    upsample = tf.nn.relu(upsample)
    return upsample
    
def max_pooling(after_conv):
    pooling = tf.nn.max_pool(after_conv, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
    
    return pooling

def get_crop_dim(conv_out, decoding_out):
    
    #print(decoding_out.get_shape())
    
    orig_h = int(conv_out.shape[1])
    orig_w = int(conv_out.shape[2])
    
    crop_size_h = int(decoding_out.get_shape()[1])
    crop_size_w = int(decoding_out.get_shape()[2])
    
    offset_h = (orig_h - crop_size_h) / 2
    offset_w = (orig_w - crop_size_w) / 2
    
    height = crop_size_h
    width = crop_size_w
    
    #print(orig_h, orig_w, offset_h, offset_w)
    
    return int(offset_h), int(offset_w), int(height), int(width)

def cropping_concat(encoding_out=None, decoding_out=None):
    crop_dim = get_crop_dim(encoding_out, decoding_out)
    crop = tf.image.crop_to_bounding_box(encoding_out, crop_dim[0],crop_dim[1],crop_dim[2],crop_dim[3])
    
    concat = tf.concat([crop, decoding_out],-1)
    return concat
    
def encoding_step():
    return enc
def decoding_step():
    return dec

In [3]:
input_dim = tf.placeholder(tf.float32, shape=[None,572,572,3])
print(input_dim)

Tensor("Placeholder:0", shape=(?, 572, 572, 3), dtype=float32)


# Encoding Step

In [4]:
# conv1 + max1
conv_1_output = conv_step(input_dim, 3, 64)
print("1_conv_layer_test : ", conv_1_output)
pool_1_output = max_pooling(conv_1_output)
print("1_pooling_layer_test : ", pool_1_output)

#conv2 + max2
conv_2_output = conv_step(pool_1_output, 3, 64*2)
print("2_conv_layer_test : ", conv_2_output)
pool_2_output = max_pooling(conv_2_output)
print("2_pooling_layer_test : ", pool_2_output)

#conv3 + max3
conv_3_output = conv_step(pool_2_output, 3, 64*4)
print("3_conv_layer_test : ", conv_3_output)
pool_3_output = max_pooling(conv_3_output)
print("3_pooling_layer_test : ", pool_3_output)

#conv4 + max4
conv_4_output = conv_step(pool_3_output, 3, 64*8)
print("3_conv_layer_test : ", conv_4_output)
pool_4_output = max_pooling(conv_4_output)
print("3_pooling_layer_test : ", pool_4_output)

#extends
extend = conv_step(pool_4_output, 3, 64*16)
print("extends : ", extend)

conv1 :  Tensor("Conv2D:0", shape=(?, 570, 570, 64), dtype=float32)
conv2 :  Tensor("Conv2D_1:0", shape=(?, 568, 568, 64), dtype=float32)
1_conv_layer_test :  Tensor("Relu:0", shape=(?, 568, 568, 64), dtype=float32)
1_pooling_layer_test :  Tensor("MaxPool:0", shape=(?, 284, 284, 64), dtype=float32)
conv1 :  Tensor("Conv2D_2:0", shape=(?, 282, 282, 128), dtype=float32)
conv2 :  Tensor("Conv2D_3:0", shape=(?, 280, 280, 128), dtype=float32)
2_conv_layer_test :  Tensor("Relu_1:0", shape=(?, 280, 280, 128), dtype=float32)
2_pooling_layer_test :  Tensor("MaxPool_1:0", shape=(?, 140, 140, 128), dtype=float32)
conv1 :  Tensor("Conv2D_4:0", shape=(?, 138, 138, 256), dtype=float32)
conv2 :  Tensor("Conv2D_5:0", shape=(?, 136, 136, 256), dtype=float32)
3_conv_layer_test :  Tensor("Relu_2:0", shape=(?, 136, 136, 256), dtype=float32)
3_pooling_layer_test :  Tensor("MaxPool_2:0", shape=(?, 68, 68, 256), dtype=float32)
conv1 :  Tensor("Conv2D_6:0", shape=(?, 66, 66, 512), dtype=float32)
conv2 :  Tens

# Decoding Step

In [5]:
#upsample and concat
up_sample1 = up_conv(extend, 2, 64*8)
concat1 = cropping_concat(encoding_out=conv_4_output, decoding_out=up_sample1)
print("concat_1 : ", concat1)
#5 conv and conv
conv_5_output = conv_step(concat1, 3, 64*4)
print("5_conv_layer_test : ", conv_5_output)

#upsample and concat
up_sample2 = up_conv(conv_5_output, 2, 64*4)
concat2 = cropping_concat(encoding_out=conv_3_output, decoding_out=up_sample2)
print("concat_2 : ", concat2)
#5 conv and conv
conv_6_output = conv_step(concat2, 3, 64*2)
print("6_conv_layer_test : ", conv_6_output)

#upsample and concat
up_sample3 = up_conv(conv_6_output, 2, 64*2)
concat3 = cropping_concat(encoding_out=conv_2_output, decoding_out=up_sample3)
print("concat_3 : ", concat3)
#5 conv and conv
conv_7_output = conv_step(concat3, 3, 64)
print("7_conv_layer_test : ", conv_6_output)

#upsample and concat
up_sample4 = up_conv(conv_7_output, 2, 64)
concat4 = cropping_concat(encoding_out=conv_1_output, decoding_out=up_sample4)
print("concat_4 : ", concat4)
#5 conv and conv
conv_8_output = conv_step(concat4, 3, 64)
print("8_conv_layer_test : ", conv_8_output)

up-conv :  Tensor("conv2d_transpose/BiasAdd:0", shape=(?, 56, 56, 512), dtype=float32)
concat_1 :  Tensor("concat:0", shape=(?, 56, 56, 1024), dtype=float32)
conv1 :  Tensor("Conv2D_10:0", shape=(?, 54, 54, 256), dtype=float32)
conv2 :  Tensor("Conv2D_11:0", shape=(?, 52, 52, 256), dtype=float32)
5_conv_layer_test :  Tensor("Relu_6:0", shape=(?, 52, 52, 256), dtype=float32)
up-conv :  Tensor("conv2d_transpose_1/BiasAdd:0", shape=(?, 104, 104, 256), dtype=float32)
concat_2 :  Tensor("concat_1:0", shape=(?, 104, 104, 512), dtype=float32)
conv1 :  Tensor("Conv2D_12:0", shape=(?, 102, 102, 128), dtype=float32)
conv2 :  Tensor("Conv2D_13:0", shape=(?, 100, 100, 128), dtype=float32)
6_conv_layer_test :  Tensor("Relu_8:0", shape=(?, 100, 100, 128), dtype=float32)
up-conv :  Tensor("conv2d_transpose_2/BiasAdd:0", shape=(?, 200, 200, 128), dtype=float32)
concat_3 :  Tensor("concat_2:0", shape=(?, 200, 200, 256), dtype=float32)
conv1 :  Tensor("Conv2D_14:0", shape=(?, 198, 198, 64), dtype=float3

# Segmenatation

In [6]:
#segmentation
#segmentation = conv_step(conv_8_output, 1, 2)
#print("segmentation : ", segmentation)

# Training

#### dataset

In [7]:
'''
tf.decode_raw(
    bytes,
    out_type,
    little_endian=True,
    name=None
)
'''
image_tensor = None
with open("./dataset/0002/Chevron_000045.jpg",'rb') as fp:
    image_tensor = tf.decode_raw(fp.read(), tf.float32)

In [8]:
print(image_tensor)

Tensor("DecodeRaw:0", shape=(?,), dtype=float32)


In [9]:
with tf.Session() as sess:
    sess.run(image_tensor) #execute init_op
    #print the random values that we sample
    print (sess.run(image_tensor))

[-1.47486124e+20  1.29320000e+04  2.35603141e-38  2.35098898e-38
  3.58732407e-43  1.28859360e+02  2.52084764e-35  4.00303168e-34
  1.62533381e-33  1.06335513e-31  2.69728581e-32  4.82931315e-28
  1.95954301e-21  2.09272050e-21  1.32230120e-19  1.41607169e-19
  5.39892163e-22  4.06689520e-14  1.65019344e-07  2.20939820e-15
  1.07241300e-08  1.66874941e-07  2.02035205e-38  1.64913684e-33
  1.07117922e-31  1.82304335e-24  5.28987895e-19  1.03723767e-08
  1.03723767e-08  1.03723767e-08  1.03723767e-08  1.03723767e-08
  1.03723767e-08  1.03723767e-08  1.03723767e-08  1.03723767e-08
  1.03723767e-08  1.03723767e-08  1.03723767e-08 -7.97487736e+00
  7.40782420e-40  5.64240158e-37  9.41371345e-38  1.03344065e-28
  1.80912354e-38  2.35099739e-38  2.36942895e-38  9.21956299e-41
  0.00000000e+00  2.35098870e-38  6.20716262e-36  1.63737069e-33
 -2.04034497e+03  1.53429850e-39  3.84985969e-37  6.16032652e-36
  3.68783921e-40  4.64694785e-38  1.50477088e-36  4.94734295e-19
  1.69453197e-27  1.83474